In [ ]:
import requests
import googlemaps
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


~~Place your google API key (with full Maps API access) in the main project directory.~~

Place your mapillary API key in a text doc "mapillary_client_token.txt" in the project directory.
https://www.mapillary.com/dashboard/developers
Just make a new registration and it should automatically give you the client token.

In [ ]:

# Reading api key + cities to pull data for
parent_folder = os.path.dirname(os.path.dirname(__name__))
#google_api_key = open(parent_folder + "google_maps_api_key.txt", "r").read()
mapillary_client_id = open(parent_folder + "mapillary_client_token.txt", "r").read()

### Openstreetmap API

API rate limit handler.

In [ ]:
### API rate limit stuff
import time
import requests
def backoff(retries):
    # To avoid API limit
    time.sleep(2 ** retries)

rate_limits = {
    "entity": 60000,
    "search": 10000,
    "tiles": 50000  # Note: This is per day
}

requests_made = {
    "entity": 0,
    "search": 0,
    "tiles": 0
}

def request_with_rate_limiting(url, headers=None, params=None, type="entity"):
    # Should exponentially slow down once we hit rate limit.
    global requests_made
    retries = 0
    while True:
        if requests_made[type] < rate_limits[type]:
            response = requests.get(url, headers=headers, params=params)
            requests_made[type] += 1
            if response.status_code == 429:  # Rate limit exceeded error
                print("Rate limit exceeded, retrying...")
                retries += 1
                backoff(retries)
            else:
                return response
        else:
            print(f"Approaching rate limit for {type}. Waiting...")
            time.sleep(60)  # Wait for 1 minute before retrying
            requests_made[type] = 0

In [ ]:
import requests
import pandas as pd
import geopandas as gpd
import os
import mercantile
from vt2geojson.tools import vt_bytes_to_geojson
import time
from shapely.geometry import Point, Polygon
from pyproj import Transformer 

country_list = pd.read_csv('./Data/country_list.csv')
european = country_list['country'].unique()

# World Mollweide (EPSG: 54009) 
urban_areas = gpd.read_file('./Data/urban/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg')

# GPD world object #'epsg:4326', (lon, lat)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world[world['name'].isin(european)]
urban_areas = urban_areas.to_crs("EPSG:4326") # Converting to 4326
# buffering
europe['geometry'] = europe['geometry'].apply(lambda x: x if x.is_valid else x.buffer(0))
urban_areas['geometry'] = urban_areas['geometry'].apply(lambda x: x if x.is_valid else x.buffer(0))
### Filtering to only europe
urban_areas_in_europe = gpd.sjoin(urban_areas, europe, how="inner", op='intersects')

dataset_file = "./Data/mapillary/dataset_mapillary.csv"

# Check if the dataset file exists
if os.path.exists(dataset_file):
    # If the file exists, load the dataset from the file
    dataset = pd.read_csv(dataset_file, index_col=0)
    n = len(dataset)
    print(f"Resuming from index {n}")
else:
    # If the file doesn't exist, create a new dataset DataFrame
    dataset = pd.DataFrame(columns=["lat", "lon", "width", "height"])
    n = 0

tile_coverage = 'mly1_public'
tile_layer = "image"
N = 2500
start_time = time.time()

### Search loop
while n < N:

    ### Should be in 4326 format
    urban_polygon = urban_areas_in_europe.sample(1).iloc[0].geometry

    ### Returned as (long, lat, long, lat)
    west, south, east, north = urban_polygon.bounds

    #print(f"West, south, east, north {west} {south} {east} {north}")

    zoom = 14
    tiles = list(mercantile.tiles(west, south, east, north, zoom))

    max_img = False
    for tidx, tile in enumerate(tiles):

        if max_img:
            break

        tile_url = f'https://tiles.mapillary.com/maps/vtp/{tile_coverage}/2/{tile.z}/{tile.x}/{tile.y}?access_token={mapillary_client_id}'
        response = request_with_rate_limiting(tile_url, type="tiles")
        #response = requests.get(tile_url)
        data = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z, layer=tile_layer)

        ### If there are no features, this skips
        for idx, feature in enumerate(data['features']):
            # Get lng,lat of each feature
            lng = feature['geometry']['coordinates'][0]
            lat = feature['geometry']['coordinates'][1]
            
            ### Checking what country we're in.
            point = gpd.GeoDataFrame([{'geometry': Point(lng, lat)}], crs='EPSG:4326')
            point_in_country = gpd.sjoin(point, world, how="inner", op='intersects')
            country_name = point_in_country.iloc[0]['name']  # Assuming 'name' is the country name column in world

            if idx == 0:
                print(f"Tile {tidx} is in {country_name}")    

            sequence_id = feature['properties']['sequence_id']
            _folder = os.path.join(parent_folder, "Data", "mapillary")

            # Request the URL of each image
            image_id = feature['properties']['id']
            url = f'https://graph.mapillary.com/{image_id}?fields=thumb_2048_url,width,height'
            headers = {'Authorization': f'OAuth {mapillary_client_id}'}
            #r = requests.get(url, headers=headers)
            r = request_with_rate_limiting(url, headers=headers, type="search")
            data = r.json()
            
            try:
                image_url = data['thumb_2048_url']
            except:
                print(f"KeyError: 'thumb_2048_url' \nNot sure why some images don't have the 2048 url, but we can just skip.")
                continue
            
            image_width = data['width']
            image_height = data['height']

            # Save each image with ID as filename to directory by sequence ID
            image_path = os.path.join(_folder, f"{n}.jpg")
            with open(image_path, 'wb') as handler:
                image_data = requests.get(image_url, stream=True).content
                handler.write(image_data)
            dataset.loc[n] = [lat, lng, image_width, image_height]
            print(f"Succesful image found for index {n} out of {N}")
            n += 1

            ### Periodically saving
            if n % 50 == 0:
                print(f"Saved dataset at n = {n}.")
                dataset.to_csv(dataset_file)

            # We don't want, e.g. 4000 images from Paris to clog the dataset.
            # So, break back to next tile, exit the tile loop, pick a new polygon.
            if idx > 10:
                print(f"Over 10 images found for this tile. Moving onto a new polygon.")
                max_img = True
                break;

dataset.to_csv(dataset_file)


For checking if your API key is set correctly

In [ ]:
import mercantile, mapbox_vector_tile, requests, json, os
from vt2geojson.tools import vt_bytes_to_geojson

def download_mapillary_images(west, south, east, north, mapillary_client_id, parent_folder, image_num):
    # Vector tile endpoint
    tile_coverage = 'mly1_public'
    tile_layer = "image"

    # Get the list of tiles with x and y coordinates which intersect our bounding box
    # MUST be at zoom level 14 where the data is available, other zooms currently not supported
    tiles = list(mercantile.tiles(west, south, east, north, 14))

    # Loop through list of tiles to get tile z/x/y to plug in to Mapillary endpoints and make request
    for tile in tiles:
        tile_url = f'https://tiles.mapillary.com/maps/vtp/{tile_coverage}/2/{tile.z}/{tile.x}/{tile.y}?access_token={mapillary_client_id}'
        response = requests.get(tile_url)
        data = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z, layer=tile_layer)
        print(response)
        # Process each feature in the tile
        for idx, feature in enumerate(data['features']):
            print(f"On feature {idx}")
            # Get lng,lat of each feature
            lng = feature['geometry']['coordinates'][0]
            lat = feature['geometry']['coordinates'][1]

            # Ensure feature falls inside bounding box since tiles can extend beyond
            if lng > west and lng < east and lat > south and lat < north:
                # Create a folder for each unique sequence ID to group images by sequence
                sequence_id = feature['properties']['sequence_id']
                sequence_folder = os.path.join(parent_folder, "Data", "mapillary", sequence_id)
                os.makedirs(sequence_folder, exist_ok=True)

                # Request the URL of each image
                image_id = feature['properties']['id']
                url = f'https://graph.mapillary.com/{image_id}?fields=thumb_2048_url'
                headers = {'Authorization': f'OAuth {mapillary_client_id}'}
                r = requests.get(url, headers=headers)
                data = r.json()
                image_url = data['thumb_2048_url']

                # Save each image with ID as filename to directory by sequence ID
                image_path = os.path.join(sequence_folder, f"{image_id}.jpg")
                with open(image_path, 'wb') as handler:
                    image_data = requests.get(image_url, stream=True).content
                    handler.write(image_data)

# Example usage
west, south, east, north = [-80.13423442840576, 25.77376933762778, -80.1264238357544, 25.788608487732198]

download_mapillary_images(west, south, east, north, mapillary_client_id, parent_folder, image_num=0)


##### Old Google Maps API code (apparently data scraping is against their TOS? Who would've thought)

In [ ]:
### Given a coord, randomly select a nearby coord to grab. This is best for picking within cities.
import geopandas as gpd
import random
import shapely.geometry as geom
def city_coords(countries, world, urban_areas):
    searching = True
    while searching:
        lat = random.uniform(-90, 90)
        lon = random.uniform(-180, 180)

        # Create a shapely Point from the latitude and longitude
        point = geom.Point(lon, lat)

        # Create a GeoDataFrame with the random point
        point_gdf = gpd.GeoDataFrame(geometry=[point], crs="EPSG:4326")
        ### Casting to 50049
        point_gdf = point_gdf.to_crs(urban_areas.crs)

        # Perform a spatial join to find the urban area that contains the point
        result_c = gpd.sjoin(point_gdf, world.to_crs(urban_areas.crs), op='within')
        
        # Check if the point is within a target country
        if not result_c.empty:
            country = result_c.iloc[0]['name']
            if country in countries:
                # Perform a spatial join to find the urban area that contains the point
                result_u = gpd.sjoin(point_gdf, urban_areas, op='within')
                
                # Check if the point is within an urban area
                if not result_u.empty:
                    urban_area = result_u.iloc[0]['eFUA_name']
                    #print(f"The point ({lat}, {lon}) is in the urban area: {urban_area}")
                    return (lat, lon, country)
                # else:
                #     print(f"The point ({lat}, {lon}) is in {country} but not in an urban area. Trying again.")
            # else:
            #     print(f"Country is {country}. Trying again.")

In [ ]:
### Converting lat-long to street address

def get_address(gmaps, coords):
    try:
        # Look up an address with reverse geocoding
        reverse_geocode_result = gmaps.reverse_geocode(coords)
        
        if reverse_geocode_result:
            geocode_components = reverse_geocode_result[0]
            coms = geocode_components['address_components']
            
            street_number = ""
            street_name = ""
            city = ""
            state = ""
            country = ""
            postal_code = ""
            
            for component in coms:
                if 'street_number' in component['types']:
                    street_number = component['long_name']
                elif 'route' in component['types']:
                    street_name = component['long_name']
                elif 'locality' in component['types']:
                    city = component['long_name']
                elif 'administrative_area_level_1' in component['types']:
                    state = component['long_name']
                elif 'country' in component['types']:
                    country = component['long_name']
                elif 'postal_code' in component['types']:
                    postal_code = component['long_name']
            
            address = f"{street_number} {street_name}, {city}, {state}, {country} {postal_code}"
            return city, address
        else:
            #print("No address found for the given coordinates.")
            return None, None
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None

In [ ]:
### Given a coord, randomly select a nearby coord to grab. This is best for picking randomly inside countries, allowing for rural picks.

import geopandas as gpd
import random
import shapely.geometry as geom

#args: countries (list of countries we want), world (GPD object)
def gen_coords(countries, world):
    searching = True
    while searching:
        lat = random.uniform(-90, 90) ### Randomly selecting a latlong
        lon = random.uniform(-180, 180)
        ### Create a shapely Point from the latitude and longitude
        point = geom.Point(lon, lat)

        ### Create a GeoDataFrame with the random point
        point_gdf = gpd.GeoDataFrame(geometry=[point], crs="EPSG:4326")

        ### Perform a spatial join to find the country that contains the point
        result = gpd.sjoin(point_gdf, world, op='within')

        if not result.empty:
            country = result.iloc[0]['name']
            if country in countries:
                #print(f"The point ({lat}, {lon}) is in {country}, which is one of the target countries.")
                return (lat, lon, country)
            # else:
            #     print(f"Country is {country}. Trying again")



Main Script

In [ ]:
### gmaps object
gmaps = googlemaps.Client(key=google_api_key)

### list of target countries
country_list = pd.read_csv('./Data/country_list.csv')
european = country_list['country'].unique()

### GPD world object
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) ### Creating here so it doesn't need to be re-created every execution

### urban areas databse (GHS)
urban_areas = gpd.read_file('./Data/urban/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg') ### A list of polygon-bounded coordinates

### Dataset file for the pipeline
dataset_file = "./Data/gmaps/dataset.csv"

# Check if the dataset file exists
if os.path.exists(dataset_file):
    # If the file exists, load the dataset from the file
    dataset = pd.read_csv(dataset_file, index_col=0)
    start_index = len(dataset)
    print(f"Resuming from index {start_index}")
else:
    # If the file doesn't exist, create a new dataset DataFrame
    dataset = pd.DataFrame(columns=["lat", "lon", "country", "address"])
    start_index = 0

### Dataset to be saved for identification of each image w/ latlong 
#dataset = pd.DataFrame(columns=["lat", "lon", "country", "address"])

N = 2500
for n in range(start_index, start_index + N):
    ### Selecting 33% rural (randomly sampled within the country) and 66% urban (bounded by urban_areas)
    searching = True
    while searching:
        if n % 3:
            lat, lon, country = city_coords(european, world, urban_areas)
        else:
            lat, lon, country = gen_coords(european, world)
        city, address = get_address(gmaps, (lat, lon))
        
        if city is None or address is None:
            continue

        ### Checking if the rounded coordinates already exist in the DataFrame. Surprising amount of duplicates.
        lat_rounded = round(lat, 6)
        lon_rounded = round(lon, 6)
        if len(dataset[(dataset['lat'].round(6) == lat_rounded) & (dataset['lon'].round(6) == lon_rounded)]) > 0:
            print(f"Coordinates ({lat_rounded}, {lon_rounded}) already exist in the dataset. Skipping...")
            continue
        
        ### Russia is gigantic. Skipping ~2/3 of all Russia hits. 
        if n % 3 and country == "Russia":
            continue

        # print(f"Address found for image {n}!")

        ### If we've found an address that exists in the database, we can try querying the API for a streetview image.
        pic_base = 'https://maps.googleapis.com/maps/api/streetview?'

        ### define the params for the picture request
        pic_params = {'key': google_api_key,
                'location': address,
                'size': "512x512"}
        
        ### Requesting data
        pic_response = requests.get(pic_base, params=pic_params)
        # print(f"API Response:")
        # print(f"Status Code: {pic_response.status_code}")
        # print(f"Headers: {pic_response.headers}")
        # print(f"Content Type: {pic_response.headers['Content-Type']}")
        # print(f"Content Length: {len(pic_response.content)} bytes")

        if pic_response.status_code == 200:
            ### If the image cannot be retrieved, gmaps will still return a blank image. The image size is typically ~6kB-9kB. 
            if len(pic_response.content) > 10000:
                image_name = f"{n}.jpg"
                image_path = parent_folder + "Data/gmaps/" + image_name
                
                with open(image_path, "wb") as file:
                    file.write(pic_response.content)
                dataset.loc[len(dataset)] = [lat, lon, country, address]
                searching = False
                print(f"Image found for image {n} in {country}.")
            # else:
            #     print(f"Image size is too small, {len(pic_response.content)} bytes - no image exists.")
        else:
            print(f"Failed to retrieve street view image for address: {address}")

    if n % 100 == 0:
        print(f"Saved dataset at n = {n}.")
        dataset.to_csv(dataset_file)

dataset.to_csv(dataset_file)